In [ ]:
from utils import *

In [ ]:
from azureml.core import Workspace
from azureml.core.authentication import InteractiveLoginAuthentication

interactive_auth = InteractiveLoginAuthentication(tenant_id="c643d8c8-4abb-47ba-808c-bf73f60161f1")

# Configure workspace
ws = Workspace.from_config(auth=interactive_auth)

In [ ]:
from azureml.core.model import InferenceConfig

env = get_env(name="sentiment-analysis", packages=["tensorflow", "transformers"])

inference_config = InferenceConfig(
    environment=env,
    source_directory="code",
    entry_script="sentiment_analysis.py",
)

In [ ]:
from azureml.core.webservice import AciWebservice

deployment_config = AciWebservice.deploy_configuration(
    cpu_cores=0.5, memory_gb=1, auth_enabled=True
)

In [ ]:
from azureml.core import Model

service = Model.deploy(
    ws,
    "sentiment-analysis",
    [],
    inference_config,
    deployment_config,
    overwrite=True,
)
service.wait_for_deployment(show_output=True)

In [ ]:
print(service.get_logs())

In [ ]:
import requests
import json
from azureml.core import Webservice

service = Webservice(workspace=ws, name="sentiment-analysis")
scoring_uri = service.scoring_uri

# If the service is authenticated, set the key or token
key, _ = service.get_keys()

# Set the appropriate headers
headers = {"Content-Type": "application/json"}
headers["Authorization"] = f"Bearer {key}"

data = {
    "query": "My ML skills are quite good."
}
data = json.dumps(data)
resp = requests.post(scoring_uri, data=data, headers=headers)
print(resp.text)

In [ ]:
scoring_uri

In [ ]:
# service.delete()